In [1]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_553169/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/12 16:38:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/12 16:38:55 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
data = [[1, '2017-01-01', 10], [2, '2017-01-02', 109], [3, '2017-01-03', 150], [4, '2017-01-04', 99], [5, '2017-01-05', 145], [6, '2017-01-06', 1455], [7, '2017-01-07', 199], [8, '2017-01-09', 188]]
stadium = pd.DataFrame(data, columns=['id', 'visit_date', 'people']).astype({'id':'Int64', 'visit_date':'datetime64[ns]', 'people':'Int64'})


df_person = spark.createDataFrame(stadium)
df_person.createOrReplaceTempView("Stadium")

stadium

,id,visit_date,people
0,1,2017-01-01,10
1,2,2017-01-02,109
2,3,2017-01-03,150
3,4,2017-01-04,99
4,5,2017-01-05,145
5,6,2017-01-06,1455
6,7,2017-01-07,199
7,8,2017-01-09,188


In [18]:
query = """

    with t as (
        select 
            *,
            sum(
                case
                    when id != previous_id + 1 then 1
                    else 0
                end
            ) over (order by id asc) as group_id
        from (
            select
                *,
                lag(id, 1, id) over(order by id asc) as previous_id
            from Stadium s 
            where people >= 100
            order by id asc
        )
    )
    
    select
        t.id,
        t.visit_date,
        t.people
    from t where group_id in (
        select
            group_id
        from t
        group by group_id
        having count(distinct id) >= 3
    ) order by t.visit_date asc
"""

In [19]:
spark.sql(query).show()

24/03/12 17:55:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/12 17:55:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/12 17:55:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/12 17:55:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/12 17:55:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/12 17:55:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/12 1

+---+-------------------+------+
| id|         visit_date|people|
+---+-------------------+------+
|  5|2017-01-05 00:00:00|   145|
|  6|2017-01-06 00:00:00|  1455|
|  7|2017-01-07 00:00:00|   199|
|  8|2017-01-09 00:00:00|   188|
+---+-------------------+------+

